In [37]:
import numpy as np
import astropy.units as u
import astropy.constants as const
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astropy.table import QTable, Table, Column, vstack
import yaml
import re
import os
import time
from pprint import pprint
from astroplan import FixedTarget, Observer, observability_table
from astroplan import AltitudeConstraint, AtNightConstraint
from astropy.coordinates import EarthLocation
from astropy.time import Time

In [ ]:

homedir = os.path.expanduser('~')
savedir = homedir + '/uasal/roman-psf-vetting/data/catalog_tables/'

from astropy.utils.data import clear_download_cache

clear_download_cache()

#Simbad.cache_location

In [3]:
with open("hd_id_list.yaml", 'r') as f:
    current_list = yaml.safe_load(f)

#print(current_list)

In [5]:
print(current_list)

['HD 358', 'HD 432', 'HD 886', 'HD 8538', 'HD 10144', 'HD 11636', 'HD 12311', 'HD 19356', 'HD 23630', 'HD 33111', 'HD 35468', 'HD 35497', 'HD 36673', 'HD 37795', 'HD 40183', 'HD 40312', 'HD 47105', 'HD 58350', 'HD 58715', 'HD 67523', 'HD 80007', 'HD 80404', 'HD 81188', 'HD 87901', 'HD 95418', 'HD 97603', 'HD 102647', 'HD 103287', 'HD 105435', 'HD 106490', 'HD 106625', 'HD 108767', 'HD 109668', 'HD 112185', 'HD 112413', 'HD 120315', 'HD 121263', 'HD 127972', 'HD 130841', 'HD 132058', 'HD 135382', 'HD 135742', 'HD 139006', 'HD 141891', 'HD 158408', 'HD 158427', 'HD 158926', 'HD 159561', 'HD 161471', 'HD 169022', 'HD 175191', 'HD 177724', 'HD 178524', 'HD 193924', 'HD 203280', 'HD 207098', 'HD 209952', 'HD 218045', 'HD 149757', 'HD 186882', 'HD 197345']


In [17]:
#pprint(EarthLocation.get_site_names())
observer = Observer.at_site('mwo')
#observer

<Observer: name='mwo',
    location (lon, lat, el)=(-118.06168400000004 deg, 34.22257980555556 deg, 1742.0000000003204 m),
    timezone=<UTC>>

In [38]:
constraints = [
    AltitudeConstraint(30 * u.deg),
    AtNightConstraint.twilight_astronomical(),
]

In [18]:
targets = []

for star in current_list:
    targets.append(FixedTarget.from_name(star))

In [27]:
semester_start_str = "2024-08-01T00:00:00"
semester_end_str = "2025-02-01T00:00:00" 
semester_start = Time(semester_start_str, format = 'isot', scale = 'utc')
semester_end = Time(semester_end_str, format = 'isot', scale = 'utc')
semester_length = semester_end - semester_start

In [36]:
dates = []

for i in range(int(semester_length.value)):
    mid_date = semester_start + (i * u.day) + (12 * u.hr)
    dates.append(mid_date)

#dates


In [45]:
tables = []

for date in dates:

    print("Processing observabilities for {}".format(date), end = '\r')

    obstable = observability_table(
        constraints,
        observer,
        targets,
        time_range = date,
        time_grid_resolution = 15 * u.min,
    )

    tables.append(obstable)

In [46]:
tables[0].show_in_notebook()

idx,target name,ever observable,always observable,fraction of time observable,time observable
,,,,,h
0,HD 358,True,False,0.1875,4.5
1,HD 432,True,False,0.22916666666666666,5.5
2,HD 886,True,False,0.16666666666666666,4.0
3,HD 8538,True,False,0.1875,4.5
4,HD 10144,False,False,0.0,0.0
5,HD 11636,True,False,0.10416666666666667,2.5
6,HD 12311,False,False,0.0,0.0
7,HD 19356,True,False,0.08333333333333333,2.0
8,HD 23630,True,False,0.041666666666666664,1.0


In [47]:
for table in tables:
    table.add_index('target name')

In [58]:

row_headers = ['id']
dtypes = [str,]
for date in dates:
    row_headers.append(date.value.split("T")[0])
    dtypes.append(float)
star_table = Table(
    data = None,
    names = row_headers,
    dtype = dtypes,
)
star_table.show_in_notebook()

idx,id,2024-08-01,2024-08-02,2024-08-03,2024-08-04,2024-08-05,2024-08-06,2024-08-07,2024-08-08,2024-08-09,2024-08-10,2024-08-11,2024-08-12,2024-08-13,2024-08-14,2024-08-15,2024-08-16,2024-08-17,2024-08-18,2024-08-19,2024-08-20,2024-08-21,2024-08-22,2024-08-23,2024-08-24,2024-08-25,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08,2024-09-09,2024-09-10,2024-09-11,2024-09-12,2024-09-13,2024-09-14,2024-09-15,2024-09-16,2024-09-17,2024-09-18,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28,2024-09-29,2024-09-30,2024-10-01,2024-10-02,2024-10-03,2024-10-04,2024-10-05,2024-10-06,2024-10-07,2024-10-08,2024-10-09,2024-10-10,2024-10-11,2024-10-12,2024-10-13,2024-10-14,2024-10-15,2024-10-16,2024-10-17,2024-10-18,2024-10-19,2024-10-20,2024-10-21,2024-10-22,2024-10-23,2024-10-24,2024-10-25,2024-10-26,2024-10-27,2024-10-28,2024-10-29,2024-10-30,2024-10-31,2024-11-01,2024-11-02,2024-11-03,2024-11-04,2024-11-05,2024-11-06,2024-11-07,2024-11-08,2024-11-09,2024-11-10,2024-11-11,2024-11-12,2024-11-13,2024-11-14,2024-11-15,2024-11-16,2024-11-17,2024-11-18,2024-11-19,2024-11-20,2024-11-21,2024-11-22,2024-11-23,2024-11-24,2024-11-25,2024-11-26,2024-11-27,2024-11-28,2024-11-29,2024-11-30,2024-12-01,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-07,2024-12-08,2024-12-09,2024-12-10,2024-12-11,2024-12-12,2024-12-13,2024-12-14,2024-12-15,2024-12-16,2024-12-17,2024-12-18,2024-12-19,2024-12-20,2024-12-21,2024-12-22,2024-12-23,2024-12-24,2024-12-25,2024-12-26,2024-12-27,2024-12-28,2024-12-29,2024-12-30,2024-12-31,2025-01-01,2025-01-02,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-11,2025-01-12,2025-01-13,2025-01-14,2025-01-15,2025-01-16,2025-01-17,2025-01-18,2025-01-19,2025-01-20,2025-01-21,2025-01-22,2025-01-23,2025-01-24,2025-01-25,2025-01-26,2025-01-27,2025-01-28,2025-01-29,2025-01-30,2025-01-31


In [59]:
for idx, star in enumerate(targets):
    name = star.name
    print("Processing {} ({})".format(name, idx), end = '\r')
    row_data = [name,]
    for table in tables:
        row_data.append(table.loc[name]['time observable'])
    star_table.add_row(row_data)


In [60]:
star_table.show_in_notebook()

idx,id,2024-08-01,2024-08-02,2024-08-03,2024-08-04,2024-08-05,2024-08-06,2024-08-07,2024-08-08,2024-08-09,2024-08-10,2024-08-11,2024-08-12,2024-08-13,2024-08-14,2024-08-15,2024-08-16,2024-08-17,2024-08-18,2024-08-19,2024-08-20,2024-08-21,2024-08-22,2024-08-23,2024-08-24,2024-08-25,2024-08-26,2024-08-27,2024-08-28,2024-08-29,2024-08-30,2024-08-31,2024-09-01,2024-09-02,2024-09-03,2024-09-04,2024-09-05,2024-09-06,2024-09-07,2024-09-08,2024-09-09,2024-09-10,2024-09-11,2024-09-12,2024-09-13,2024-09-14,2024-09-15,2024-09-16,2024-09-17,2024-09-18,2024-09-19,2024-09-20,2024-09-21,2024-09-22,2024-09-23,2024-09-24,2024-09-25,2024-09-26,2024-09-27,2024-09-28,2024-09-29,2024-09-30,2024-10-01,2024-10-02,2024-10-03,2024-10-04,2024-10-05,2024-10-06,2024-10-07,2024-10-08,2024-10-09,2024-10-10,2024-10-11,2024-10-12,2024-10-13,2024-10-14,2024-10-15,2024-10-16,2024-10-17,2024-10-18,2024-10-19,2024-10-20,2024-10-21,2024-10-22,2024-10-23,2024-10-24,2024-10-25,2024-10-26,2024-10-27,2024-10-28,2024-10-29,2024-10-30,2024-10-31,2024-11-01,2024-11-02,2024-11-03,2024-11-04,2024-11-05,2024-11-06,2024-11-07,2024-11-08,2024-11-09,2024-11-10,2024-11-11,2024-11-12,2024-11-13,2024-11-14,2024-11-15,2024-11-16,2024-11-17,2024-11-18,2024-11-19,2024-11-20,2024-11-21,2024-11-22,2024-11-23,2024-11-24,2024-11-25,2024-11-26,2024-11-27,2024-11-28,2024-11-29,2024-11-30,2024-12-01,2024-12-02,2024-12-03,2024-12-04,2024-12-05,2024-12-06,2024-12-07,2024-12-08,2024-12-09,2024-12-10,2024-12-11,2024-12-12,2024-12-13,2024-12-14,2024-12-15,2024-12-16,2024-12-17,2024-12-18,2024-12-19,2024-12-20,2024-12-21,2024-12-22,2024-12-23,2024-12-24,2024-12-25,2024-12-26,2024-12-27,2024-12-28,2024-12-29,2024-12-30,2024-12-31,2025-01-01,2025-01-02,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-07,2025-01-08,2025-01-09,2025-01-10,2025-01-11,2025-01-12,2025-01-13,2025-01-14,2025-01-15,2025-01-16,2025-01-17,2025-01-18,2025-01-19,2025-01-20,2025-01-21,2025-01-22,2025-01-23,2025-01-24,2025-01-25,2025-01-26,2025-01-27,2025-01-28,2025-01-29,2025-01-30,2025-01-31
0,HD 358,4.5,5.0,5.5,5.5,5.5,5.5,5.5,5.5,5.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.5,6.5,6.5,6.5,6.5,6.5,6.5,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.5,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.5,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.0,2.5,2.5,2.5,2.5,2.5,2.5,2.5,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.5,1.5,1.5,1.5,1.5,1.5
1,HD 432,5.5,6.0,6.5,6.5,6.5,6.5,6.5,6.5,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.5,7.5,7.5,7.5,7.5,7.5,7.5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.5,9.5,9.5,9.5,9.5,9.5,9.5,10.0,10.0,10.0,9.5,9.5,9.5,9.5,9.5,9.5,9.5,9.5,9.0,9.0,9.0,9.0,9.0,9.0,9.5,9.0,9.0,9.0,9.0,9.0,9.0,9.0,9.0,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,5.5,5.5,5.5,5.5,5.5,5.5,5.5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,4.5,4.5,4.5,4.5,4.5,4.5,4.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.5,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,2.5,2.5,2.5,2.5,2.5
2,HD 886,4.0,4.0,4.5,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.5,5.5,5.5,5.5,5.5,5.5,5.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.5,6.5,6.5,6.5,6.5,6.5,7.0,7.0,7.5,7.5,7.5,7.5,7.5,7.5,7.5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.5,8.0,8.0,8.0,8.0,8.0,8.0,8.0,7.5,7.5,7.5,7.5,7.5,7.5,7.5,8.0,7.5,7.5,7.5,7.5,7.5,7.5,7.5,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.5,6.0,6.0,6.0,6.0,6.0,6.0,6.0

In [61]:
star_simple_table = Table()

star_simple_table.add_column(
    star_table['id'],
    name = 'id',
)

In [67]:
# For > 1 hour visible
visible_mins = [1, 2, 3, 4]

for min in visible_mins:
    name = "Days visible for >{}hr".format(min)
    vals = []
    for row in star_table:
        print(row[1:])
        days_above_min = np.where(row[1:] >= min)[0]
        n_days = len(days_above_min)
        vals.append(n_days)
    star_simple_table.add_column(
        vals,
        name = name,
    )
    

In [68]:
star_simple_table.show_in_notebook()

idx,id,Days visible for >1hr,Days visible for >2hr,Days visible for >3hr,Days visible for >4hr
0,HD 358,21,17,13,11
1,HD 432,21,17,13,11
2,HD 886,21,17,14,11
3,HD 8538,21,17,15,11
4,HD 10144,20,17,15,11
5,HD 11636,20,17,15,11
6,HD 12311,19,18,15,11
7,HD 19356,19,18,15,11
8,HD 23630,19,17,14,11
9,HD 33111,19,17,14,11
